<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/7_2_%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 7-1 장에서 살펴본 인공신경망은 히든레이어가 1개인 신경망이었다.
# 입력층과 출력층 사이에 층을 더 추가해서 심층 신경망을 만들어보자.

from tensorflow import keras
(train_input, train_target), (test_input, test_target) =\
keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 # 1. 표준화
train_scaled = train_scaled.reshape(-1, 28*28) # 2. 각 그림을 1차원 벡터로 만듬.

# 3. 훈련 세트에서 다시 훈련/검증 세트로 나눔.
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [4]:
# 이제 인공신경망 모델에 층을 추가해보자.

# 입력층: (n_samples, 28*28)
# h1: (28*28, 100) -> 뉴런 수가 100개인 히든레이어 h1. 활성화함수는 시그모이드/렐루 같은걸 사용할 수 있음.
# 출력층: (100, 10) -> 다중 분류이므로 소프트맥스 함수를 활성화 함수로 적용. 이진분류는 시그모이드함수.

# 회귀의 경우 출력층에 활성화함수를 적용하지 않는다. 그 이유는 분류는 활성화함수로 확률로의 변환이 필요하지만, 회귀는 
# 임의의 숫자를 추론하는 모델이므로 적용하지 않는다.

# 여기서 은닉층에 활성화함수를 사용하는 이유를 살펴보면 아래 예시를 확인해보자.
# a * 3 - 1 = b
# 2 * b + 3 = c 라고 했을 때, 활성화 함수를 적용하지 않는 경우라고 볼 수 있다.
# 결국 c = 6 * a + 1 이되며 b가 하는 일이 없는거나 마찬가지다.
# 이러한 선형 방정식을 적절히 비선형으로 뒤틀어주어야 한다. 그래야 이전 히든레이어가 존재하는 의미가 생기기 때문이다.

# a * 3 - 1 = b
# log(b) = k
# 2 * k + 3 = c
# 처럼.

# 자 그럼 이제 은닉층의 활성화함수로 시그모이드 함수를, 출력층의 활성화 함수로 소프트맥스 함수를 사용한 인공신경망을 만들어보자.
# 히든레이어부터 출력층까지 만든다.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(28*28,))
dense2 = keras.layers.Dense(10, activation='softmax')

# dense1, dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어보자.
model = keras.Sequential([dense1, dense2])
# keras는 모델의 summary() 메서드로 층에 대한 유용한 정보를 보여준다.
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# dense1, dense2 객체를 만들어서 keras.Sequential 메서드에 리스트로 전달하여 신경망을 만들었지만,
# 일반적으로 아래 방법으로 케라스로 심층신경망을 만든다.

model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden1'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 Mnist 모델')

model.summary()
# dense1, dense2 같은 객체를 따로 저장해서 쓸 일이 없기 때문에 위 처럼 하는게 일반적이다.
# 참고로 모델 파라미터 개수가 78500 인 이유는 784 입력층 X 히든레이어 뉴런 100개 곱인 78400개에
# 히든레이어 절편 100개 를 더했기 때문이다.

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 하지만, 이런 히든레이어가 하나면 상관없지만, 만약 10개가 있다고 할 경우, 모델 선언이 너무 길어지고 복잡해진다.
# 아래 방법이 가장 널리 사용된다.

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='softmax', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 이제, 만든 케라스 DNN 모델을 갖고 훈련해보자.
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6081 - accuracy: 0.7623
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5910 - accuracy: 0.7739
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5760 - accuracy: 0.7839
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5638 - accuracy: 0.7887
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5531 - accuracy: 0.7961


In [16]:
# sigmoid 말고 이미지 분류에서 뛰어난 성능을 낼 수 있는 활성화 함수인 'ReLU' 에 대해 알아보자.

# 1. sigmoid 활성화 함수의 한계
# 시그모이드 함수는 절편 0.5를 기준으로 오른쪽으로 갈 수록, 왼 쪽으로 갈수록 활성화 함수의 값이 각각 1, 0에 가깝다.
# 그래서 심층 신경망일 수록 은닉층을 거칠 때 마다 출력되는 값의 변화가 거의 없어서 모델파라미터 업데이트가 느린 문제가 있다.

# 이를 보완하기 위해 렐루(ReLU) 라는 활성 함수가 제안되었다.
# ReLU(z) = max(0, z)
# 입력이 0이하면 0, 그 이상이면 그대로 출력하는 활성화 함수다.

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 1. 입력층 바로 뒤 flatten 층을 둔다. 은닉층에 들어가기 전 Reshape이 아닌 flatten 층을 넣어준다.
model.add(keras.layers.Dense(100, activation='relu')) # 2. 은닉층
model.add(keras.layers.Dense(10, activation='softmax')) # 3. 출력층

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               78500     
                                                                 
 dense_11 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 위 Summary() 해석
# flatten 층은 모델 파라미터가 없다. 단순히 입력되는 데이터를 벡터로 shape 변환이 임무.
# flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있다는 장점이 있다.
# 입력 데이터에 대한 전처리 과정을 가능한 한 모델에 포함시키는 것이 케라스 API 의 철학 중 하나다.

(train_input, train_target), (test_input, test_targer) =\
keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, random_state=42, test_size=0.2)

In [18]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5285 - accuracy: 0.8127
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3900 - accuracy: 0.8601
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3533 - accuracy: 0.8740
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3343 - accuracy: 0.8802
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3161 - accuracy: 0.8876


In [20]:
# sigmoid 활성함수를 신경망의 활성화 함수로 적용한 성능 대비, 렐루 함수를 적용했을 때가 성능이 더 높은 걸 알 수 있다.
# 검증 세트에서도 확인해보자.

model.evaluate(val_scaled, val_target)

# 검증 세트에서도 훈련세트 대비 비슷한 성능을 보임을 알 수 있다.
# epochs=5 로 5번의 에포크 동안 훈련했는데, 더 훈련하지 않을 이유가 없다.
# 그 전에 인공신경망의 하이퍼파라미터에 대해 좀 더 알아보자.

375/375 [==============================] - 0s 964us/step - loss: 0.3590 - accuracy: 0.8752


[0.3590386211872101, 0.875249981880188]

# 옵티마이저

> 모델 파라미터는 인공신경망이 학습하는 인공신경망의 가중치로, 이는 사람이 관여할 사항이 아니다.

> 하이퍼 파라미터는 인공신경망이 학습하는 대상이 아닌, 개발자가 지정하는 설정값이다. 

> 하이퍼 파라미터 예시: ```epoch수```, ```은닉층 뉴런수```, ```층의종류(DNN, CNN, ...)```, ```은닉층 개수```, ```미니배치사이즈```, ```경사하강알고리즘(=옵티마이저) 종류(RMSprop, SGD, ...)```, ```학습률(learning rate)```

In [21]:
# 여러 옵티마이저를 활용해보자.
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 'sgd' 로 optimizer 매개변수를 전달해도 되지만, 아래처럼 명시적으로 SGD 객체를 전달해줘도 된다.
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

# SGD 클래스의 디폴트 학습률 (learning_rate) 가 0.01 이라고 할 때, 이를 바꾸고 싶다면 learning_rate 매개변수에 저장한다.
# sgd = keras.optimizers.SGD(learning_rate=0.1)

In [24]:
# 옵티마이저의 종류 2가지
# 1. 기본 경사하강 옵티마이저 (lr 고정)
# 1-1) SGD (momentum 디폴트값은 0)
# 1-2) momentum (SGD 클래스에서 momentum > 0 일 때 이전의 그래디언트를 가속도로 활용)
# 1-3) nesterov momentum (SGD 클래스에서 nesterov=True 로 바꾸면 네스테로프모멘텀 최적화를 수행. 모멘텀최적화를 2번 반복하는 방식)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
# 일반적으로 nesterov 모멘텀 최적화가 기본 확률적 경사하강법보다 더 나은 성능을 제공..

# 2. 적응적 학습률 옵티마이저 (lr 변동)
# 모델이 최적점에 가까이 갈 수록, 학습률을 낮추어 안정적으로 성능 수렴할 수 있는 옵티마이저.
# 아래 세 알고리즘 모두 디폴트 학습률은 0.001이다. (1e-3)
# 2-1) RMSprop 
rmsprop = keras.optimizers.RMSprop()
model.compile(loss='sparse_categorical_crossentropy', optimizer=rmsprop, metrics='accuracy')
# 2-2) Adagrad
adagrad = keras.optimizers.Adagrad()
model.compile(loss='sparse_categorical_crossentropy', optimizer=adagrad, metrics='accuracy')

# 2-3) Adam : RMSprop + momentum
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5228 - accuracy: 0.8160
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3930 - accuracy: 0.8591
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3516 - accuracy: 0.8729
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3237 - accuracy: 0.8821
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3066 - accuracy: 0.8870


In [25]:
# RMSprop 을 옵티마이저로 지정했을 때나 Adam 이나 훈련세트의 점수는 거의 비슷하다.
# 검증 세트에 대해서도 확인해보자.
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 889us/step - loss: 0.3341 - accuracy: 0.8799


[0.33405858278274536, 0.8799166679382324]

In [26]:
# 마찬가지로 비슷하다.